In [4]:
import pandas as pd
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import scrapy_user_agents

In [5]:
class QuotesLogin(scrapy.Spider):
    # Name of your spider
    name = "login"

    # Url to start your spider from
    start_urls = [
        'https://quotes.toscrape.com/login',
    ]

    # Callback function that will be called when starting your spider
    def parse(self, response):
        return scrapy.FormRequest.from_response(
            response,
            formdata = {'username':'iuzf', 'password':'azmjfpazo'},
            callback = self.after_login
        )
    
    

    def after_login(self, response):
        quotes = response.css('div.quote')
        for quote in quotes:
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('span small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }

        
        try:
            next_page = response.css('li.next a').attrib["href"]
        except KeyError:
            logging.info("No next page !")
        else:
            yield response.follow(next_page, callback=self.after_login)
            

In [6]:
ROTATING_PROXY_LIST = [
    '38.91.106.204:40103',
    '147.135.255.62:8242'
]

filename = "4_quotesauthentication.json"

if filename in os.listdir('src/'):
    os.remove('src/' + filename)
    
#Créer un CrawlerProcess
process = CrawlerProcess(settings = {
    'USER_AGENT':'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"}},
        "AUTOTHROTTLE_ENABLED":True,
            "DOWNLOADER_MIDDLEWARES": {
    'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
    'scrapy_user_agents.middlewares.RandomUserAgentMiddleware': 400
},
    "ROTATING_PROXY_LIST" : ROTATING_PROXY_LIST
})


#Lancer le crawler utilisant la spider
process.crawl(QuotesLogin)
process.start()

2025-02-24 10:43:39 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2025-02-24 10:43:39 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.0.15 3 Sep 2024), cryptography 43.0.0, Platform Windows-11-10.0.26100-SP0
2025-02-24 10:43:39 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-24 10:43:39 [py.warnings] WARNING: c:\Users\darla\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu

In [7]:
pd.read_json("src/" + filename)

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]
